# Загрузка библиотек

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
import re
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from joblib import dump, load

# Загрузка данных

In [2]:
reviews_train = []
for line in open('data//train_data.txt', 'r'):
    reviews_train.append(line.strip())
    
reviews_test = []
for line in open('data//test_data.txt', 'r'):
    reviews_test.append(line.strip())

In [3]:
# сразу сделаем предварительную очистку данных: избавимся от знаков и сделаем все буквы строчными

REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

def preprocess_reviews(reviews):
    reviews = [REPLACE_NO_SPACE.sub("", line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(" ", line) for line in reviews]
    
    return reviews

reviews_train_clean = preprocess_reviews(reviews_train)
reviews_test_clean = preprocess_reviews(reviews_test)

# Целевые признаки

In [4]:
# Статус комменатрия: положительный/отрицательный
target = [1 if i < 12500 else 0 for i in range(25000)]

# Подготовка

Функция для обучения модели и вывода метрик

In [5]:
def model_fit_score(model):
    print(model)
    # обучение
    model.fit(X_train, y_train)
    print('точность на валидационных данных: ', accuracy_score(y_val, model.predict(X_val)))
    # обучим на всех данных
    model.fit(X, target)
    print('точность на тестовых данных :', accuracy_score(target, model.predict(X_test)))

Будем тестировать две модели:
- логистическую регрессию
- метод опорных векторов 

Обе эти модели хорошо работют с разреженными данными.

In [6]:
lr = LogisticRegression(random_state=10)
svc = LinearSVC()

# Поиск решения

## N-граммы

Попробуем вместо отдельных слов использовать n-граммы. А также исключим стоп-слова.

In [7]:
# вариант с ngram_range=(1, 2)
cv1 = CountVectorizer(binary=True, ngram_range=(1, 2), stop_words='english')
# обучаем на трейн-данных
cv1.fit(reviews_train_clean)
# векторизуем трейн и тест данные
X = cv1.transform(reviews_train_clean)
X_test = cv1.transform(reviews_test_clean)
# Разделение на трейн и валидацию
X_train, X_val, y_train, y_val = train_test_split(X, target, train_size = 0.8, random_state=10)

In [8]:
model_fit_score(lr)

LogisticRegression(random_state=10)
точность на валидационных данных:  0.8908
точность на тестовых данных : 0.8854


In [9]:
model_fit_score(svc)

LinearSVC()
точность на валидационных данных:  0.884
точность на тестовых данных : 0.87988


C:\Users\i_a_f\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [10]:
# вариант с ngram_range=(1, 3)
cv2 = CountVectorizer(binary=True, ngram_range=(1, 3), stop_words='english')
# обучаем на трейн-данных
cv2.fit(reviews_train_clean)
# векторизуем трейн и тест данные
X = cv2.transform(reviews_train_clean)
X_test = cv2.transform(reviews_test_clean)
# Разделение на трейн и валидацию
X_train, X_val, y_train, y_val = train_test_split(X, target, train_size = 0.8, random_state=10)

In [11]:
model_fit_score(lr)

LogisticRegression(random_state=10)
точность на валидационных данных:  0.884
точность на тестовых данных : 0.8838


In [12]:
model_fit_score(svc)

LinearSVC()


C:\Users\i_a_f\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


точность на валидационных данных:  0.8874
точность на тестовых данных : 0.8834


# TF-IDF 

Попробуем использовать TF-IDF статистику вместо слов/n-грамм.

In [7]:
tfidf_vectorizer1 = TfidfVectorizer()
# обучаем на трейн-данных
tfidf_vectorizer1.fit(reviews_train_clean)
# векторизуем трейн и тест данные
X = tfidf_vectorizer1.transform(reviews_train_clean)
X_test = tfidf_vectorizer1.transform(reviews_test_clean)
# Разделение на трейн и валидацию
X_train, X_val, y_train, y_val = train_test_split(X, target, train_size = 0.8, random_state=10)

In [8]:
model_fit_score(lr)

LogisticRegression(random_state=10)
точность на валидационных данных:  0.8874
точность на тестовых данных : 0.8824


In [15]:
model_fit_score(svc)

LinearSVC()
точность на валидационных данных:  0.897
точность на тестовых данных : 0.8794


In [16]:
# вариант с удалением стоп-слов
tfidf_vectorizer2 = TfidfVectorizer(stop_words='english')
# обучаем на трейн-данных
tfidf_vectorizer2.fit(reviews_train_clean)
# векторизуем трейн и тест данные
X = tfidf_vectorizer2.transform(reviews_train_clean)
X_test = tfidf_vectorizer2.transform(reviews_test_clean)
X_train, X_val, y_train, y_val = train_test_split(X, target, train_size = 0.8, random_state=10)

In [17]:
model_fit_score(lr)

LogisticRegression(random_state=10)
точность на валидационных данных:  0.8912
точность на тестовых данных : 0.87924


In [18]:
model_fit_score(svc)

LinearSVC()
точность на валидационных данных:  0.8902
точность на тестовых данных : 0.86704


# Подбор гиперпараметров

Наилучший результат на тестовых данных показала модель логистической регрессии на n-граммах (1,2). С TF-IDF результат чуть похуже, но в качестве финальной модели выберу все-таки его, так как дамп этой модели получается гораздо более "легким" и работает гораздо шустрее. Попробуем подобрать к нему оптимальные гиперпараметры.

In [10]:
# заново подготовим данные  - векторизуем трейн и тест данные
X = tfidf_vectorizer1.transform(reviews_train_clean)
X_test = tfidf_vectorizer1.transform(reviews_test_clean)
# Разделение на трейн и валидацию
X_train, X_val, y_train, y_val = train_test_split(X, target, train_size = 0.8, random_state=10)

In [11]:
for c in [0.01, 0.1, 0.5, 1]:
    model_fit_score(LogisticRegression(random_state=10, C=c))
    print()

LogisticRegression(C=0.01, random_state=10)
точность на валидационных данных:  0.7968
точность на тестовых данных : 0.79612

LogisticRegression(C=0.1, random_state=10)
точность на валидационных данных:  0.8544
точность на тестовых данных : 0.85048

LogisticRegression(C=0.5, random_state=10)
точность на валидационных данных:  0.8818
точность на тестовых данных : 0.87844

LogisticRegression(C=1, random_state=10)
точность на валидационных данных:  0.8874
точность на тестовых данных : 0.8824



Наилучшее качество при С=1

# Финальная модель

In [13]:
# финальная модель с оптимальными параметрами
final_model = LogisticRegression(C=1, random_state=10)
# обучим на всех данных
final_model.fit(X, target)
print('точность на тестовых данных :', accuracy_score(target, final_model.predict(X_test)))

точность на тестовых данных : 0.8824


In [15]:
# сохранение моделей
dump(tfidf_vectorizer1, './saved_models/vectorizer_pos_neg.joblib')
dump(final_model, './saved_models/model_pos_neg.joblib')

['./saved_models/model_pos_neg.joblib']

# Тестирование

In [2]:
vectorizer = load('./saved_models/vectorizer_pos_neg.joblib')
model_pos_neg = load('./saved_models/model_pos_neg.joblib')

In [19]:
test1 = ['The movie is awesome! Hooked, the second day I think about it. \
        Real life with its unexpected twists and turns. Excellent acting.\
        To say an interesting, intriguing story is to say nothing. \
        I havent seen such deep character development in a long time. \
        I highly recommend watching the movie!']
test1 = vectorizer.transform(test1)
'POSITIVE' if (model_pos_neg.predict(test1)[0] == 1) else 'NEGATIVE'

'POSITIVE'

In [20]:
test2= ['Its just a nightmare. The movie is so crazy! The main characters are incredibly vile. \
        The plot is terrible. I was disgusted to watch. Whats the point, whats the moral']
test2 = vectorizer.transform(test2)
'POSITIVE' if (model_pos_neg.predict(test2)[0] == 1) else 'NEGATIVE'

'NEGATIVE'

In [21]:
test3= ['The best!']
test3 = vectorizer.transform(test3)
'POSITIVE' if (model_pos_neg.predict(test3)[0] == 1) else 'NEGATIVE'

'POSITIVE'